In [1]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import concatenate
from keras.layers import Input
from keras.layers import Dropout
from math import sqrt
from matplotlib import pyplot
import numpy as np
from numpy import array

# date-time parsing function for loading the dataset
def parser(x):
    return datetime.strptime('190'+x, '%Y-%m')
 
# convert time series into supervised learning problem
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)
 
# transform series into train and test sets for supervised learning
def prepare_data(series, n_test, n_lag, n_seq):
    # extract raw values
    raw_values = series.values
    # transform data to be stationary
    diff_series = difference(raw_values, 1)
    diff_values = diff_series.values
    diff_values = diff_values.reshape(len(diff_values), 1)
    # rescale values to -1, 1
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_values = scaler.fit_transform(diff_values)
    scaled_values = scaled_values.reshape(len(scaled_values), 1)
    # transform into supervised learning problem X, y
    supervised = series_to_supervised(scaled_values, n_lag, n_seq)
    return supervised
 
# fit an LSTM network to training data
def fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons):
    # reshape training into [samples, timesteps, features]
    X, y = train[:, 0:n_lag], train[:, n_lag:]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    # design network
    model = Sequential()
    model.add(LSTM(128, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True, return_sequences=True))
    model.add(LSTM(32, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
    model.add(RepeatVector(3))
    model.add(LSTM(128, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True, return_sequences=True))
    model.add(LSTM(32, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(y.shape[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    # fit network
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=n_batch, verbose=0, shuffle=False)
        model.reset_states()
    return model

# make one forecast with an LSTM,
def forecast_lstm(model, X, n_batch):
    # reshape input pattern to [samples, timesteps, features]
    X = X.reshape(1, 1, len(X))
    # make forecast
    forecast = model.predict(X, batch_size=n_batch)
    # convert to array
    return [x for x in forecast[0, :]]
 
# evaluate the persistence model
def make_forecasts(model, n_batch, train, test, n_lag, n_seq):
    forecasts = list()
    for i in range(len(test)):
        X, y = test[i, 0:n_lag], test[i, n_lag:]
        # make forecast
        forecast = forecast_lstm(model, X, n_batch)
        # store the forecast
        forecasts.append(forecast)
    return forecasts

# make one forecast with an LSTM,
def forecast_encoder(model, X, n_batch):
    # reshape input pattern to [samples, timesteps, features]
    X = X.reshape(1, 1, len(X))
    # make forecast
    forecast, state_h, state_c = model.predict(X, batch_size=n_batch)
    # convert to array
    return state_c
 
# evaluate the persistence model
def make_forecasts_encoder(model, n_batch, train, test, n_lag, n_seq):
    states = list()
    for i in range(len(test)):
        X, y = test[i, 0:n_lag], test[i, n_lag:]
        # make forecast
        state_c = forecast_encoder(model, X, n_batch)
        # store the forecast
        states.append(state_c)
    return states

# invert differenced forecast
def inverse_difference(last_ob, forecast):
    # invert first forecast
    inverted = list()
    inverted.append(forecast[0] + last_ob)
    # propagate difference forecast using inverted first value
    for i in range(1, len(forecast)):
        inverted.append(forecast[i] + inverted[i-1])
    return inverted
 
# inverse data transform on forecasts
def inverse_transform(series, forecasts, scaler, n_test):
    inverted = list()
    for i in range(len(forecasts)):
        # create array from forecast
        forecast = array(forecasts[i])
        forecast = forecast.reshape(1, len(forecast))
        # invert scaling
        inv_scale = scaler.inverse_transform(forecast)
        inv_scale = inv_scale[0, :]
        # invert differencing
        index = len(series) - n_test + i - 1
        last_ob = series.values[index]
        inv_diff = inverse_difference(last_ob, inv_scale)
        # store
        inverted.append(inv_diff)
    return inverted
 
# evaluate the RMSE for each forecast time step
def evaluate_forecasts(test, forecasts, n_lag, n_seq):
    for i in range(n_seq):
        actual = [row[i] for row in test]
        predicted = [forecast[i] for forecast in forecasts]
        rmse = sqrt(mean_squared_error(actual, predicted))
        smape = mean_absolute_percentage_error(actual, predicted)
        print('t+%d RMSE: %f' % ((i+1), rmse))
        print('t+%d SMAPE: %f' % ((i+1), smape))

# SMAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / ((y_true + y_pred) / 2))) * 100

# plot the forecasts in the context of the original dataset
def plot_forecasts(series, forecasts, n_test):
    # plot the entire dataset in blue
    pyplot.plot(series.values)
    # plot the forecasts in red
    for i in range(len(forecasts)):
        off_s = len(series) - n_test + i - 1
        off_e = off_s + len(forecasts[i]) + 1
        xaxis = [x for x in range(off_s, off_e)]
        yaxis = [series.values[off_s]] + forecasts[i]
        pyplot.plot(xaxis, yaxis, color='red')
    # show the plot
    pyplot.show()



Using TensorFlow backend.


In [5]:
# configure
n_lag = 12
n_seq = 1
n_test = 22
n_epochs = 1500
n_batch = 1
n_neurons = 1

In [6]:
# load dataset
series = read_csv('shampoo_sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

In [7]:
display(prepare_data(series,n_test,n_lag,n_seq))

,var1(t-12),var1(t-11),var1(t-10),var1(t-9),var1(t-8),var1(t-7),var1(t-6),var1(t-5),var1(t-4),var1(t-3),var1(t-2),var1(t-1),var1(t)
12,-0.639992,0.013926,-0.405945,0.112866,-0.189773,0.122428,-0.171066,-0.272501,-0.431303,0.747246,-0.766784,-0.105799,-0.326959
13,0.013926,-0.405945,0.112866,-0.189773,0.122428,-0.171066,-0.272501,-0.431303,0.747246,-0.766784,-0.105799,-0.326959,0.111203
14,-0.405945,0.112866,-0.189773,0.122428,-0.171066,-0.272501,-0.431303,0.747246,-0.766784,-0.105799,-0.326959,0.111203,0.122012
15,0.112866,-0.189773,0.122428,-0.171066,-0.272501,-0.431303,0.747246,-0.766784,-0.105799,-0.326959,0.111203,0.122012,-0.481189
16,-0.189773,0.122428,-0.171066,-0.272501,-0.431303,0.747246,-0.766784,-0.105799,-0.326959,0.111203,0.122012,-0.481189,0.256703
17,0.122428,-0.171066,-0.272501,-0.431303,0.747246,-0.766784,-0.105799,-0.326959,0.111203,0.122012,-0.481189,0.256703,-0.394305
18,-0.171066,-0.272501,-0.431303,0.747246,-0.766784,-0.105799,-0.326959,0.111203,0.122012,-0.481189,0.256703,-0.394305,0.181875
19,-0.272501,-0.431303,0.747246,-0.766784,-0.105799,-0.326959,0.111203,0.122012,-0.481189,0.256703,-0.394305,0.181875,-0.197672
20,-0.431303,0.747246,-0.766784,-0.105799,-0.326959,0.111203,0.122012,-0.481189,0.256703,-0.394305,0.181875,-0.197672,0.406776
21,0.747246,-0.766784,-0.105799,-0.326959,0.111203,0.122012,-0.481189,0.256703,-0.394305,0.181875,-0.197672,0.406776,-0.793806


In [13]:
#prepare train-test sets
df = prepare_data(series,n_test,n_lag,n_seq)
# display(df)
values = df.values
encoder_train = values[0:7]
encoder_test_bnn_train = values[7:15]
bnn_test = values[15:23]
state_y_train = encoder_test_bnn_train[:,-1]
state_y_test = bnn_test[:,-1]

[[-0.27250052 -0.43130326  0.74724589 -0.76678445 -0.10579921 -0.32695905
   0.11120349  0.12201206 -0.48118894  0.25670339 -0.39430472  0.18187487
  -0.197672  ]
 [-0.43130326  0.74724589 -0.76678445 -0.10579921 -0.32695905  0.11120349
   0.12201206 -0.48118894  0.25670339 -0.39430472  0.18187487 -0.197672
   0.40677614]
 [ 0.74724589 -0.76678445 -0.10579921 -0.32695905  0.11120349  0.12201206
  -0.48118894  0.25670339 -0.39430472  0.18187487 -0.197672    0.40677614
  -0.79380586]
 [-0.76678445 -0.10579921 -0.32695905  0.11120349  0.12201206 -0.48118894
   0.25670339 -0.39430472  0.18187487 -0.197672    0.40677614 -0.79380586
   0.1827063 ]
 [-0.10579921 -0.32695905  0.11120349  0.12201206 -0.48118894  0.25670339
  -0.39430472  0.18187487 -0.197672    0.40677614 -0.79380586  0.1827063
  -0.15152775]
 [-0.32695905  0.11120349  0.12201206 -0.48118894  0.25670339 -0.39430472
   0.18187487 -0.197672    0.40677614 -0.79380586  0.1827063  -0.15152775
   0.2779048 ]
 [ 0.11120349  0.12201206

In [16]:
lstm = fit_lstm(encoder_train, n_lag, n_seq, n_batch, n_epochs, n_neurons)

In [17]:
def extract_embedding(model, train, n_lag, n_batch):
    X, y = train[:, 0:n_lag], train[:, n_lag:]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    input1 = Input(batch_shape=(n_batch, X.shape[1], X.shape[2]))
    x = LSTM(128, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True, return_sequences=True)(input1)
    x, state_h, state_c = LSTM(32, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True, return_state=True)(x)
    encoder = Model(inputs=input1, outputs=[x, state_h, state_c])
    return encoder

In [31]:
encoder = extract_embedding(lstm, encoder_train, n_lag, n_batch)
states = make_forecasts_encoder(encoder, n_batch, encoder_train, encoder_test_bnn_train, n_lag, n_seq)
for i in range(len(states)):
    states[i] = np.append(states[i], state_y_train[i])

In [59]:
# fit an BNN network to training data
def fit_bnn(train, n_input, n_batch, nb_epoch, n_neurons):
    X = [row[0:-1] for row in train]
    y = [row[-1] for row in train]
    X = np.array(X)
    # Hardcode
    input1 = Input(shape=(n_input,))
    x = Dense(128, activation="tanh")(input1)
    x = Dropout(0.5)(x, training=True)
    x = Dense(64, activation="tanh")(x)
    x = Dropout(0.5)(x, training=True)
    x = Dense(16, activation="tanh")(x)
    x = Dropout(0.5)(x, training=True)
    x = Dense(1)(x)
    bnn = Model(inputs=[input1], outputs=[x])
    bnn.compile(loss='mean_squared_error', optimizer='adam')
    # fit network
    for i in range(nb_epoch):
        bnn.fit(X, y, epochs=1, verbose=0, shuffle=False)
    return bnn

In [57]:
states = np.array(states)
bnn = fit_bnn(states, 32, n_batch, n_epochs, n_neurons)

(8, 32)


In [58]:
bnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 32)                0         
_________________________________________________________________
dense_53 (Dense)             (None, 128)               4224      
_________________________________________________________________
dropout_34 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_54 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_35 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_55 (Dense)             (None, 16)                1040      
_________________________________________________________________
dropout_36 (Dropout)         (None, 16)                0         
__________

In [ ]:
# make one forecast with an bnn,
def forecast_bnn(model, X):
    # reshape input pattern to [samples, timesteps, features]
    X = X.reshape(1, 1, len(X))
    # make forecast
    forecast = model.predict(X)
    # convert to array
    return [x for x in forecast[0, :]]
 
# evaluate the persistence model
def make_forecasts_bnn(model, train):
    forecasts = list()
    X = [row[0:-1] for row in train]
    y = [row[-1] for row in train]
    X = np.array(X)
    for i in range(len(test)):
        X, y = test[i, 0:n_lag], test[i, n_lag:]
        # make forecast
        forecast = forecast_lstm(model, X, n_batch)
        # store the forecast
        forecasts.append(forecast)
    return forecasts

In [ ]:
# make forecasts
forecasts = make_forecasts(model, n_batch, train, test, n_lag, n_seq)
# inverse transform forecasts and test
forecasts = inverse_transform(series, forecasts, scaler, n_test)
actual = [row[n_lag:] for row in test]
actual = inverse_transform(series, actual, scaler, n_test)
# evaluate forecasts
evaluate_forecasts(actual, forecasts, n_lag, n_seq)
# plot forecasts
plot_forecasts(series, forecasts, n_test)

In [9]:
# prepare data
scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)
print train
# fit model
model = fit_lstm(train, n_lag, n_seq, n_batch, n_epochs, n_neurons)

ValueError: too many values to unpack